In [6]:
import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical
from torch.optim import Adam
import numpy as np
import gym
from gym.spaces import Discrete, Box
import math

In [ ]:
# make environment, check spaces, get obs / act dims
env = gym.make('CartPole-v1', render_mode="human")

obs, _ = env.reset()  
env.render()

assert isinstance(env.observation_space, Box), \
    "This example only works for envs with continuous state spaces."
assert isinstance(env.action_space, Discrete), \
    "This example only works for envs with discrete action spaces."

obs_dim = env.observation_space.shape[0]
n_acts = env.action_space.n




In [ ]:
# torch.distributions.Categorical usage

# Define a probability distribution
probs = torch.tensor([0.1, 0.3, 0.6])  # Must sum to 1
dist = torch.distributions.Categorical(probs)

# Sample from the distribution
sample = dist.sample()
print("Sampled action:", sample.item())

# Get log probability of a specific action
log_prob = dist.log_prob(torch.tensor(1))

# math.log(0.3) = -1.2039728043259361
print("Log probability of action 1:", log_prob.item())

indices = torch.arange(len(probs))  # Tensor of indices [0, 1, 2]
log_probs = dist.log_prob(indices)
print("Log probabilities of all actions:", log_probs)

# Entropy of the distribution
entropy = dist.entropy()
print("Entropy:", entropy.item())


Sampled action: 2
Log probability of action 1: -1.2039728164672852
Log probabilities of all actions: tensor([-2.3026, -1.2040, -0.5108])
Entropy: 0.897945761680603


In [7]:
math.log(0.3)

-1.2039728043259361

In [11]:
p = [0.1, 0.3, 0.6]

# Calculate the entropy term: p_i * log(p_i)
entropy_term = sum(p_i * math.log(p_i) for p_i in p)

entropy_term = -entropy_term  # Negate the sum to get the entropy

print(entropy_term)

0.8979457248567797


In [12]:
import torch.nn as nn
import torch.nn.functional as F

class PolicyNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        logits = self.fc(x)
        return torch.distributions.Categorical(logits=logits)  # Convert logits to a categorical distribution

# Example usage
policy_net = PolicyNetwork(4, 2)  # Example: 4 input features, 2 possible actions
state = torch.rand(4)  # Example input
dist = policy_net(state)
action = dist.sample()
log_prob = dist.log_prob(action)


tensor([-0.6433, -0.7456], grad_fn=<SubBackward0>)